In [15]:
import requests
import urllib
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm import tqdm
import folium 
from geopy.geocoders import GoogleV3, Bing
from yaml import safe_load

In [16]:
with open('api_keys.yaml', 'r') as f:
    API_KEY = safe_load(f)['bing']

In [18]:
url = r'https://nationalmap.gov/epqs/pqs.php?'

CITIES_DATA_PATH = 'data/Cities.csv'
geolocator = Bing(api_key=API_KEY)

In [19]:
def get_coordinates_elevation(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

    df['elev_meters'] = elevations

In [20]:
cities_df = pd.read_csv(CITIES_DATA_PATH)
cities_df.head(5)
cities_df['full_name'] = cities_df['City'] + ', ' + cities_df['State'] + ', U.S.'
#

In [21]:

coordinates = []
for city in tqdm(cities_df['full_name'].values):
    try:
        location = geolocator.geocode(city)

        city_coordinates = (location.latitude, location.longitude)
        coordinates.append(city_coordinates)
    except:
        coordinates.append(None)



100%|██████████| 472/472 [01:39<00:00,  4.73it/s]


In [22]:
cities_df['coordinates'] = coordinates
cities_df.coordinates.isna().value_counts()

False    472
Name: coordinates, dtype: int64

In [25]:
for row in cities_df.head(5):
    print(row)

CityID
City
State
full_name
coordinates


In [23]:
us_map = folium.Map(location=[48, -102], zoom_start=3)

# Check whether cities are indeed in the us; 
for city_coords in cities_df.coordinates.values:
    try:
        latitude, longitude = city_coords
        folium.Marker([latitude, longitude], popup).add_to(us_map)
    except:
        print(city_coords)



In [24]:

us_map